In [17]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()

# Unsloth is an open source library developed to make finetuning LLMs a LOT faster
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True



model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",      #"unsloth/tinyllama-bnb-4bit"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:66: UserWarning: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.2
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Apache 2 free license: http://github.com/unslothai/unsloth


You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length,
)

Unsloth 2024.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
# !pip install datasets

In [6]:
from datasets import load_dataset
datasets = load_dataset("roneneldan/TinyStories", split ="train[:30]")
EOS_TOKEN = tokenizer.eos_token
def formatting_func(example):
  return example["text"] + EOS_TOKEN


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [7]:
for row in datasets[:2]["text"]:
  print("===========")
  print(row)

One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.

Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."

Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.
Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.

One day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were falling. B

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
# from transformers.utils import logging
# logging.set_verbosity_info()


trainer = SFTTrainer(
    model = model,
    train_dataset = datasets,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.05,
        max_grad_norm = 1.0,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),

)


Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
trainer_stats = trainer.train()

Step,Training Loss
1,0.300300


In [10]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
[
"Once upon a time, there was a kingdom, far far away,"
 ]*1, return_tensors = "pt").to("cuda")


generation_kwargs = dict(
  inputs,
  streamer = text_streamer,
  max_new_tokens = 256,
  use_cache = True,
)

thread = Thread (target = model.generate, kwargs = generation_kwargs)
thread.start()


length = 0
for j, new_text in enumerate(text_streamer):
  if j == 0:
    wrapped_text = textwrap.wrap(new_text, width = max_print_width)
    length= len(wrapped_text[-1])
    wrapped_text = "\n".join(wrapped_text)
    print(wrapped_text, end = "")
  else:
    length += len(new_text)
    if length >= max_print_width:
      length = 0
      print()
    print(new_text, end = "")
  pass
pass

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Once upon a time, there was a kingdom, far faraway, where the people were happy and the king 
was wise. The king had a daughter, who was beautiful and kind. She was loved by all.

One day, the 
king decided to marry his daughter to a prince from a neighboring kingdom. The prince was handsome and 
brave, and the king was sure that he would make a good husband for his daughter.

The prince was 
overjoyed when he heard the news of his marriage. He set out on a journey to the kingdom of his bride, and 
on the way, he met a witch. The witch told him that he would never be able to marry the princess if 
he did not have a magic ring.

The prince was disappointed, but he knew that he had to find a way to 
get the ring. He traveled to the land of the dwarves, and there he found a ring that was said to have 
magical powers.

The prince took the ring and returned to the kingdom of his bride. He presented the ring 
to the king, and the king was pleased. He gave his blessing to the marriage, an